## Random Agent

In [135]:
import lyra_graphtool_test as lgtool
from utils import ORIGN, BASIC_SITE, SITE1, SITE2,  SITE3, Node, Graph, Worker
import random
from copy import deepcopy

# 1. Load Graph From JSON

In [136]:
json_filename = "Test_Graph_Slightly_Off_the_Beaten_Path/Test_Graph_Slightly_Off_the_Beaten_Path.json"
graph = Graph(json_filename=json_filename)

Graph Initialised Successfully!


# 2. Random Agent Class

In [137]:
class RandomWalkAgent() :

    def __init__(self, graph : Graph) -> None:
        self.graph: Graph = graph
        self.schedule: list = []
        self.BUDGET = 10000.00
        self.REWARD = 0.00
        self.COST = 0.00
        self.actions = ["HIRE_1", "HIRE_2", "HIRE_3", "MOVE", "EXTRACT"]
        self.worker_list: list[Worker] = []

    def hire_worker(self, action, ts):
        schedule = None
        if (self.BUDGET >= self.graph.get_worker_cost(1) and action == "HIRE_1"):
            schedule = (1, self.graph.get_Origin(), ts + 1)
            self.COST += self.graph.get_worker_cost(1)
            self.BUDGET -= self.graph.get_worker_cost(1)
            self.worker_list.append(Worker(1, self.graph.get_Origin(), self.graph.get_worker_cost(1), ts))
        if (self.BUDGET >= self.graph.get_worker_cost(2) and action == "HIRE_2"):
            schedule= (2, self.graph.get_Origin(), ts + 1)
            self.COST += self.graph.get_worker_cost(2)
            self.BUDGET -= self.graph.get_worker_cost(2)
            self.worker_list.append(Worker(2, self.graph.get_Origin(), self.graph.get_worker_cost(2), ts))
        if (self.BUDGET >= self.graph.get_worker_cost(3) and action == "HIRE_3"):
            schedule=(3, self.graph.get_Origin(), ts + 1)
            self.COST += self.graph.get_worker_cost(3)
            self.BUDGET -= self.graph.get_worker_cost(3)
            self.worker_list.append(Worker(3, self.graph.get_Origin(), self.graph.get_worker_cost(3), ts))
        return schedule

    def get_schedule(self):
        for ts in range(20):
            print(ts)
            schedule = []
            if (len(self.worker_list) == 0): # Case where its empty
                print("Choosing to hire worker first")
                while(len(self.worker_list) == 0):
                    chosen_action = random.choice(["HIRE_1", "HIRE_2", "HIRE_3"])
                    schedule1 = self.hire_worker(chosen_action, ts)
                    print(schedule1)
                    schedule.append(schedule1)
            else : 
                print(self.schedule)
                for worker_state in self.schedule[-1]:
                    worker_type, worker_location, worker_ts = worker_state
                    if (worker_ts > ts):
                        schedule.append(deepcopy(worker_state))
                    else:
                        if (not self.graph.is_reward_site_by_node(node=worker_location)):
                            assert(worker_ts == ts)
                            # deduct cost
                            if (self.BUDGET < self.graph.get_worker_cost(worker_type)):
                                continue
                            self.COST += self.graph.get_worker_cost(worker_type)
                            self.BUDGET -= self.graph.get_worker_cost(worker_type)
                            neighbours = self.graph.get_adjacent_nodes_by_Node(worker_location)
                            next_state: Node = random.choice(neighbours)
                            if (self.graph.is_reward_site_by_node(next_state) and not next_state.is_accessed_before() and worker_type >= next_state.get_type()):
                                next_state.access()
                                schedule.append((worker_type, next_state, ts + next_state.get_acquire_time() + 1))
                            else :
                                schedule.append((worker_type,next_state, ts + 1))
                        else:
                            # worker is in reward site
                            assert(worker_ts == ts)
                            if (self.BUDGET >= self.graph.get_worker_cost(worker_type) * worker_location.get_acquire_time()):
                                self.BUDGET -= self.graph.get_worker_cost(worker_type) * worker_location.get_acquire_time()
                                self.COST += self.graph.get_site_reward(worker_location.get_type() - 1)
                                self.REWARD += worker_location.get_reward()
                                print("Get reward")
                                if (self.BUDGET < self.graph.get_worker_cost(worker_type)):
                                    continue
                                neighbours = self.graph.get_adjacent_nodes_by_Node(worker_location)
                                next_state: Node = random.choice(neighbours)
                                if (self.graph.is_reward_site_by_node(next_state) and not next_state.is_accessed_before() and worker_type >= next_state.get_type()):
                                    next_state.access()
                                    schedule.append((worker_type, next_state, ts + next_state.get_acquire_time() + 1))
                                else :
                                    schedule.append((worker_type,next_state, ts + 1))
                            else:
                                # fire the worker
                                continue
                if (random.choice([True, False]) and self.BUDGET >= self.graph.get_worker_cost(1)):
                    while(True):
                        schedule1 = self.hire_worker(random.choice(["HIRE_1", "HIRE_2", "HIRE_3"]), ts)
                        if (schedule1 is None):
                            new_worker = None
                        else:
                            schedule.append(schedule1)
                            break
            self.schedule.append(schedule)
        print(self.schedule)
        return self.schedule
    
    def get_profit(self):
        print(f'Rewards: {self.REWARD}, COST: {self.COST}, $ LEFT: {self.BUDGET}')
        return self.REWARD - self.COST


In [138]:
rwa = RandomWalkAgent(graph)

In [139]:
rwa.get_schedule()

0
Choosing to hire worker first
(3, ((1, 4) TYPE : 1), 1)
1
[[(3, ((1, 4) TYPE : 1), 1)]]
Get reward
2
[[(3, ((1, 4) TYPE : 1), 1)], [(3, ((0, 3) TYPE : 0), 2), (1, ((1, 4) TYPE : 1), 2)]]
Get reward
3
[[(3, ((1, 4) TYPE : 1), 1)], [(3, ((0, 3) TYPE : 0), 2), (1, ((1, 4) TYPE : 1), 2)], [(3, ((1, 4) TYPE : 1), 3), (1, ((0, 3) TYPE : 0), 3)]]
Get reward
4
[[(3, ((1, 4) TYPE : 1), 1)], [(3, ((0, 3) TYPE : 0), 2), (1, ((1, 4) TYPE : 1), 2)], [(3, ((1, 4) TYPE : 1), 3), (1, ((0, 3) TYPE : 0), 3)], [(3, ((0, 3) TYPE : 0), 4), (1, ((1, 4) TYPE : 1), 4), (2, ((1, 4) TYPE : 1), 4)]]
Get reward
Get reward
5
[[(3, ((1, 4) TYPE : 1), 1)], [(3, ((0, 3) TYPE : 0), 2), (1, ((1, 4) TYPE : 1), 2)], [(3, ((1, 4) TYPE : 1), 3), (1, ((0, 3) TYPE : 0), 3)], [(3, ((0, 3) TYPE : 0), 4), (1, ((1, 4) TYPE : 1), 4), (2, ((1, 4) TYPE : 1), 4)], [(3, ((1, 4) TYPE : 1), 5), (1, ((0, 3) TYPE : 0), 5), (2, ((0, 3) TYPE : 0), 5)]]
Get reward
6
[[(3, ((1, 4) TYPE : 1), 1)], [(3, ((0, 3) TYPE : 0), 2), (1, ((1, 4) TYP

[[(3, ((1, 4) TYPE : 1), 1)],
 [(3, ((0, 3) TYPE : 0), 2), (1, ((1, 4) TYPE : 1), 2)],
 [(3, ((1, 4) TYPE : 1), 3), (1, ((0, 3) TYPE : 0), 3)],
 [(3, ((0, 3) TYPE : 0), 4),
  (1, ((1, 4) TYPE : 1), 4),
  (2, ((1, 4) TYPE : 1), 4)],
 [(3, ((1, 4) TYPE : 1), 5),
  (1, ((0, 3) TYPE : 0), 5),
  (2, ((0, 3) TYPE : 0), 5)],
 [(3, ((2, 4) TYPE : 0), 6),
  (1, ((1, 4) TYPE : 1), 6),
  (2, ((1, 4) TYPE : 1), 6)],
 [(3, ((3, 5) TYPE : 4), 7),
  (1, ((2, 4) TYPE : 0), 7),
  (2, ((1, 5) TYPE : 0), 7),
  (1, ((1, 4) TYPE : 1), 7)],
 [(3, ((2, 6) TYPE : 0), 8),
  (1, ((1, 4) TYPE : 1), 8),
  (2, ((2, 6) TYPE : 0), 8),
  (1, ((1, 5) TYPE : 0), 8)],
 [(3, ((3, 7) TYPE : 2), 10),
  (1, ((0, 3) TYPE : 0), 9),
  (2, ((1, 5) TYPE : 0), 9),
  (1, ((2, 6) TYPE : 0), 9)],
 [(3, ((3, 7) TYPE : 2), 10),
  (1, ((1, 4) TYPE : 1), 10),
  (2, ((2, 6) TYPE : 0), 10),
  (1, ((1, 5) TYPE : 0), 10)],
 [(3, ((2, 6) TYPE : 0), 11),
  (1, ((2, 4) TYPE : 0), 11),
  (2, ((3, 7) TYPE : 2), 11),
  (1, ((1, 6) TYPE : 4), 11)]

In [140]:
rwa.get_profit()

Rewards: 23000.0, COST: 28500.0, $ LEFT: 0.0


-5500.0